In [1]:
%pip install --upgrade --quiet  annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install langchain
!pip install langchain_community
!pip install sentence-transformers
!pip install python-dotenv
!pip install langchain_core

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Annoy

embeddings_func = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [5]:
loader = CSVLoader(file_path='/content/data.csv')
documents = loader.load()

In [6]:
len(documents)

1000

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

1164

In [10]:
vector_store_from_docs = Annoy.from_documents(documents, embeddings_func)

In [114]:
from dotenv import load_dotenv

load_dotenv()

True

In [117]:
! echo HUGGINGFACEHUB_API_TOKEN="token" >> .env

In [119]:
from langchain.llms.huggingface_hub import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={"temperature":0.1, "max_length":200}
)

In [19]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [128]:
from langchain.prompts.prompt import PromptTemplate
template_string="""
<system>\
Your a helpful assisstant. You gives the relative output for user question by using content.
</system>

content: {context}

<user> {user}</user>

answer:
"""
prompt = PromptTemplate.from_template(template=template_string)


In [158]:
query = "women tshirt"
docs = vector_store_from_docs.similarity_search_with_score(query,k=5)

In [138]:
# motorbike_emb = embeddings_func.embed_query("women tshirt")
# docs=vector_store_from_docs.similarity_search_by_vector(motorbike_emb, k=3)

In [154]:
docs

[(Document(page_content='\ufeffsize: XXL\nbrand: Global Desi\ndominant_material: viscose\nactual_color: Blue | Green\nproduct_type: Top\nbody: Blue and green printed a-line top, has a tie-up neck, three-quarter sleeves, tie-ups at the sleeves\nThe model (height 5\'8") is wearing a size S\nLook chic all year long with this trendsetting top by Global Desi.  Whether you\'re grabbing a quick lunch or heading out to a nice dinner, pair this blue piece with dark trousers and a trendy clutch for a truly versatile look.\n100% viscoseMachine wash\nproduct_details: Blue and green printed a-line top, has a tie-up neck, three-quarter sleeves, tie-ups at the sleeves\ntype: Clothing/Women/Tops/Global Desi/More by Global Desi\nvariant_price: 1019\nvariant_compare_at_price: 1699\nideal_for: Women\nis_in_stock: In Stock', metadata={'source': '/content/data.csv', 'row': 136}),
  1.0370367765426636),
 (Document(page_content='\ufeffsize: XS\nbrand: GERUA\ndominant_material: Viscose Rayon\nactual_color: Re

In [159]:
chain=prompt|llm|output_parser
result=chain.invoke({"context":docs, "user":query})

In [160]:
result

'\n<system>Your a helpful assisstant. You gives the relative output for user question by using content.\n</system>\n\ncontent: [(Document(page_content="\\ufeffsize: L\\nbrand: AKKRITI BY PANTALOONS\\ndominant_material: rayon\\nactual_color: White | Purple\\nproduct_type: Top\\nbody: White and purple woven top, has a V-neck, sleeveless, a short placket with mock button-and-loop closures, a contrast print on the front, a solid back, and a slightly flared hemline\\nThe model (height 5\'8\'\') is wearing a size S\\n100% rayon Hand-wash cold\\nAdd a touch of elegance to your look with this top. Team it up with a pair of jeans and flats for a modish ensemble.\\nproduct_details: White and purple woven top, has a V-neck, sleeveless, a short placket with mock button-and-loop closures, a contrast print on the front, a solid back, and a slightly flared hemline\\ntype: Clothing/Women/Tops/AKKRITI BY PANTALOONS/More by AKKRITI BY PANTALOONS\\nvariant_price: 602\\nvariant_compare_at_price: 899\\nide

In [163]:
# Split the result by the delimiter "Document("
data_split = result.split("Document(")[1:]

# Extract relevant information from each split
parsed_data = []
for idx, item in enumerate(data_split, 1):
    # Extract the page content
    content_start_index = item.find("page_content=") + len("page_content=")
    content_end_index = item.find(", metadata=")
    page_content = item[content_start_index:content_end_index].strip()

    # Replace special characters with newline characters
    page_content = page_content.replace("\\n", "\n")

    # Append to parsed_data list
    parsed_data.append((idx, page_content))

# Print the extracted data
for idx, data in parsed_data:
    print(f"{idx}. {data}\n")


1. "\ufeffsize: L
brand: AKKRITI BY PANTALOONS
dominant_material: rayon
actual_color: White | Purple
product_type: Top
body: White and purple woven top, has a V-neck, sleeveless, a short placket with mock button-and-loop closures, a contrast print on the front, a solid back, and a slightly flared hemline
The model (height 5'8'') is wearing a size S
100% rayon Hand-wash cold
Add a touch of elegance to your look with this top. Team it up with a pair of jeans and flats for a modish ensemble.
product_details: White and purple woven top, has a V-neck, sleeveless, a short placket with mock button-and-loop closures, a contrast print on the front, a solid back, and a slightly flared hemline
type: Clothing/Women/Tops/AKKRITI BY PANTALOONS/More by AKKRITI BY PANTALOONS
variant_price: 602
variant_compare_at_price: 899
ideal_for: Women
is_in_stock: Out of Stock"

2. '\ufeffsize: L
brand: AKKRITI BY PANTALOONS
dominant_material: Viscose
actual_color: Teal
product_type: Top
body: Teal printed woven 